# IBM DataScience - Capstone Project - Part 1 - Sven De Smit

## Business understanding

Belgium is a small country in central Europe with approx. 11 million habitants. The capital of Belgium is Brussels, which is also the capital of Europe.

Despite having merely the size of a big city, Belgium is a divided country. There are three communities: a French-speaking community in the South (approx. 4 million), a Dutch-speaking community in the North (approx. 7 million), and a very small German-speaking community.

The fact that there is a political difference is proven with each election. Most people also live with the idea that there are also social and cultural differences, but hard proof is not immediately available.
Many companies – especially the multi-nationals located in Brussels – are working with a mix of employees from both parts of the country. Those companies already found a way to deal with the language difference, but the HR departments of those companies would also want to work with the cultural and differences – if they exist.

The hypothesis that we want to validate: 

_**There are important social and cultural differences between northern (Dutch-speaking) and southern (French-speaking) cities in Belgium.**_

## Analytical Approach

We believe that a natural way to characterize a city - and the people that live in that city - is by the **popularity of its venues**. For example by tallying the amount of parks, bars, restaurants or universities it has relative to all other types of venues, one can get a sense of the cultural and social character of a city.

Therefore, if we could lay our hands on data w.r.t. what the popular venues are in each city, we could use **clustering techniques** to classify cities into categories. These categories can then be visualised on a map to get an idea about the **geographical dispersion** of the categories. If there is a difference between categories mainly appearing in the south and categories mainly appearing in the north, we have proven our hypothesis.

## Data Acquisition

In [1]:
#!conda install -c conda-forge beautifulsoup4 --yes

In [2]:
#!conda install -c conda-forge geopy --yes

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

### Belgian cities data

Get HTML page

In [4]:
cities_page = 'https://nl.wikipedia.org/wiki/Tabel_van_Belgische_gemeenten'

In [5]:
req = requests.get(cities_page)
print(req.encoding)
req.status_code

UTF-8


200

In [6]:
#req.text

Parse HTML page with BeautifulSoup

In [7]:
soup = BeautifulSoup(req.text, 'html.parser')
#print(soup.prettify())

Find the HTML table and load the elements in matrix

In [8]:
data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
#print(table)
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols])

Create dataframe from matrix

In [9]:
# first row is empty. Code will break if this changes!
df_cities = pd.DataFrame(data[1:])
print('Number of cities:',df_cities.shape[0])
df_cities.head(5)

Number of cities: 581


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,Antwerpen,118.682,383.557,526.396,446.525,525.935,100,323,444,376,443,"204,51",2.572,"88,2",Antwerpen
1,2,Gent,128.828,222.895,254.216,224.180,262.219,100,173,197,174,204,"156,18",1.679,"102,3",Oost-Vlaanderen
2,3,Charleroi,54.694,198.837,233.737,200.827,202.267,100,364,427,367,370,"102,08",1.982,"73,0",Henegouwen
3,4,Luik,89.943,206.384,231.502,185.639,197.327,100,229,257,206,219,"69,39",2.844,"81,4",Luik
4,5,Brussel,129.680,218.623,184.838,133.859,181.726,100,169,143,103,140,"32,61",5.573,"70,0",Brussel


In [10]:
df_cities.drop(columns=[2,3,4,5,7,8,9,10],inplace=True)
df_cities.columns = ['ID','Name','Habitants','Index','Acreage','HabitantsPerSquareKm','ProsperityIndex','Province']
df_cities

,ID,Name,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province
0,1,Antwerpen,525.935,443,"204,51",2.572,"88,2",Antwerpen
1,2,Gent,262.219,204,"156,18",1.679,"102,3",Oost-Vlaanderen
2,3,Charleroi,202.267,370,"102,08",1.982,"73,0",Henegouwen
3,4,Luik,197.327,219,"69,39",2.844,"81,4",Luik
4,5,Brussel,181.726,140,"32,61",5.573,"70,0",Brussel
5,6,Schaarbeek,133.309,2.146,"8,14",16.377,"65,4",Brussel
6,7,Anderlecht,119.714,2.007,"17,74",6.747,"63,8",Brussel
7,8,Brugge,118.325,194,"138,40",855,"111,7",West-Vlaanderen
8,9,Namen,110.779,260,"175,69",631,"99,9",Namen
9,10,Leuven,101.624,288,"56,63",1.794,"113,6",Vlaams-Brabant


In [29]:
df_cities.dtypes

ID                      object
Name                    object
Habitants               object
Index                   object
Acreage                 object
HabitantsPerSquareKm    object
ProsperityIndex         object
Province                object
Latitude                object
Longitude               object
BoundingBox             object
dtype: object

### Location of the cities

In order to visualize the cities on a map of Belgium we need their geo-location (latitude-longitude). This location can be retrieved using the geopy package in Python. 

Inspect the data returned for an example city

In [11]:
geoloc = Nominatim(user_agent='svendesmit')
loc = geoloc.geocode('Belgium, Lokeren',timeout=2000)
loc.raw

{'place_id': 198103138,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 1325552,
 'boundingbox': ['51.0712889', '51.1646724', '3.896651', '4.0339723'],
 'lat': '51.1042159',
 'lon': '3.9911114',
 'display_name': 'Lokeren, Sint-Niklaas, Oost-Vlaanderen, Vlaanderen, 9160, België / Belgique / Belgien',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.6099319557480131,
 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png'}

Get the location data for all cities and store the data elements in lists

In [12]:
import numpy as np
lat_list = []
lng_list = []
bb_list = []
pc_list = []
for city in df_cities['Name']:
    #print(city)
    lookupstr = 'Belgium, {}'.format(city)
    loc = geoloc.geocode(lookupstr,timeout=5000)
    #print(loc.raw['display_name'].split(','))
    if(loc != None):
        lat_list.append(loc.raw['lat'])
        lng_list.append(loc.raw['lon'])
        bb_list.append(loc.raw['boundingbox'])
        #pc_list.append(loc.raw['display_name'].split(',')[4])
        #print(loc.raw['display_name'].split(',')[4])
    else:
        lat_list.append(np.NaN)
        lng_list.append(np.NaN)
        bb_list.append(np.NaN)
        #pc_list.append(np.NaN)
    

Create pandas dataframe from lists

In [13]:
df_cities['Latitude'] = lat_list
df_cities['Longitude'] = lng_list
df_cities['BoundingBox'] = bb_list
print('Number of cities:',df_cities.shape[0])
df_cities.head()

Number of cities: 581


,ID,Name,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,BoundingBox
0,1,Antwerpen,525.935,443,"204,51",2.572,"88,2",Antwerpen,51.2211097,4.3997081,"[51.1432868, 51.3776412, 4.2175769, 4.4979684]"
1,2,Gent,262.219,204,"156,18",1.679,"102,3",Oost-Vlaanderen,51.0538286,3.7250121,"[50.9795422, 51.187946, 3.5797616, 3.849325]"
2,3,Charleroi,202.267,370,"102,08",1.982,"73,0",Henegouwen,50.4120332,4.4436244,"[50.3527894, 50.4925149, 4.3474458, 4.5075571]"
3,4,Luik,197.327,219,"69,39",2.844,"81,4",Luik,50.6451381,5.5734203,"[50.5610182, 50.6881981, 5.5233883, 5.675257]"
4,5,Brussel,181.726,140,"32,61",5.573,"70,0",Brussel,50.8465573,4.351697,"[50.6865573, 51.0065573, 4.191697, 4.511697]"


In [14]:
df_cities.to_csv('cities_dataset.csv',index=False)

### Names of the cities in the three country-languages 

Many cities in Belgium have different names in the different languages. The following file that is published by the federal government contains the city names for the three languages.

In [15]:
# KBO-codes-identificatie.xls
NIS_codes_df = pd.read_excel('KBO-codes-identificatie.xls', sheet_name='NIS codes' , index_col=0, skiprows=[0])
NIS_codes_df.columns = ['Frans','Nederlands','Duits','Begin','Einde']
NIS_codes_df.head(10)


,Frans,Nederlands,Duits,Begin,Einde
CODE,,,,,
12041,NaN,Puurs-Sint-Amands,NaN,01.01.2019,31.12.9999
44083,NaN,Deinze,NaN,01.01.2019,31.12.9999
44084,NaN,Aalter,NaN,01.01.2019,31.12.9999
44085,NaN,Lievegem,NaN,01.01.2019,31.12.9999
45068,NaN,Kruisem,NaN,01.01.2019,31.12.9999
51067,Enghien,Edingen,NaN,01.01.2019,31.12.9999
51068,Silly,Opzullik,NaN,01.01.2019,31.12.9999
51069,Lessines,Lessen,NaN,01.01.2019,31.12.9999
55085,Seneffe,-,-,01.01.2019,31.12.9999


In [16]:
NIS_codes_df.to_csv('cities_multilang_dataset.csv',index=False)

### City venues data from Foursqaure 

A good source of information for venues all over the world is Foursquare. Foursquare offers a Places API to gain real-time access to Foursquare’s global database of rich venue data and user content. One can easily find popular venues in a city or a location by using the explore function in the Place API.

GET https://api.foursquare.com/v2/venues/explore

Load credentials from file

Remark: You have to register to get a client_id and secret to be able to access that API. By using the free subscription you can make a limited number of API calls per day. 

In [17]:
df_credentials = pd.read_csv('credentials.txt',header=None)
CLIENT_ID = df_credentials[1][0]
CLIENT_SECRET = df_credentials[1][1]
VERSION = '20180605' # Foursquare API version

Inspect the data returned for an example city

In [18]:
near = 'Lokeren, Belgium'
radius = 2000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            near, 
            radius, 
            LIMIT)

result = requests.get(url).json()
result["response"]['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4c5093d9bd099521bed1525e',
   'name': 'De Donkere Wolk',
   'location': {'address': 'Torenstraat 14',
    'lat': 51.105140425790516,
    'lng': 3.991317566213707,
    'labeledLatLngs': [{'label': 'display',
      'lat': 51.105140425790516,
      'lng': 3.991317566213707}],
    'postalCode': '9160',
    'cc': 'BE',
    'city': 'Lokeren',
    'state': 'Oost-Vlaanderen',
    'country': 'België',
    'formattedAddress': ['Torenstraat 14', '9160 Lokeren', 'België']},
   'categories': [{'id': '4bf58dd8d48988d116941735',
     'name': 'Bar',
     'pluralName': 'Bars',
     'shortName': 'Bar',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4c5093d9bd099521bed1525e-0'},
 {'reasons': {'cou

Function that calls the Foursquare API and stores the results in a dataframe
This dataframe contains a row for each venue found for all Belgian cities

In [19]:
def getNearbyVenues(names, radius_list):
    
    cities_found=[]
    cities_not_found = []
    cities_total = []
    venues_list=[]
    for name, radius in zip(names, radius_list):
        print(name,radius)
        cities_total.append(name)
        near = '{}, Belgium'.format(name) 
        #print(near)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            near, 
            radius, 
            LIMIT)
            
        # make the GET request
        #print(requests.get(url).json())
        found = False
        results = requests.get(url).json()
        if 'groups' in results["response"]:
            if(results["response"]['groups'] != None):
                if(results["response"]['groups'][0]['items'] != None):
                    item_list = results["response"]['groups'][0]['items']
                    if(item_list != None and len(item_list) > 0):
                        found = True
                    #print(results)
                    #print(' ')
                    # return only relevant information for each nearby venue
                    venues_list.append([(
                        name, 
                        v['venue']['name'], 
                        v['venue']['location']['lat'], 
                        v['venue']['location']['lng'],
                        v['venue']['categories'][0]['icon']['prefix'],
                        v['venue']['categories'][0]['name']) for v in item_list])
        
        if(found == True):
            cities_found.append(name)
        else:
            cities_not_found.append(name)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Category Class','Venue Category']
    
    print('Total cities: {} , {}'.format(len(cities_total),cities_total))
    print('Cities found: {} , {}'.format(len(cities_found),cities_found))
    print('Total cities: {} , {}'.format(len(cities_not_found),cities_not_found))

    return(nearby_venues)

Calculate the search radius for each city, based on the size of the city (Acreage)

In [32]:
radius_list = df_cities['Acreage']
radius_list.replace({',': '.'}, regex=True,inplace=True)
radius_list = radius_list.astype(float)/70*1000
#radius_list.head()

Execute the venue lookup using the Foursquare API  

In [21]:
be_venues = getNearbyVenues(df_cities['Name'],radius_list)
be_venues

Antwerpen 2921.5714285714284
Gent 2231.1428571428573
Charleroi 1458.2857142857142
Luik 991.2857142857143
Brussel 465.85714285714283
Schaarbeek 116.28571428571429
Anderlecht 253.4285714285714
Brugge 1977.1428571428573
Namen 2509.8571428571427
Leuven 809.0
Sint-Jans-Molenbeek 84.14285714285714
Bergen 2093.285714285714
Elsene 90.57142857142857
Mechelen 931.2857142857142
Aalst 1116.0
Ukkel 327.2857142857143
La Louvière 917.7142857142856
Hasselt 1460.5714285714284
Sint-Niklaas 1197.142857142857
Kortrijk 1143.142857142857
Oostende 538.8571428571428
Doornik 3053.5714285714284
Genk 1255.0
Seraing 504.8571428571429
Roeselare 854.1428571428571
Moeskroen 572.5714285714286
Sint-Lambrechts-Woluwe 103.14285714285714
Vorst 89.28571428571429
Verviers 472.42857142857144
Jette 72.0
Sint-Gillis 36.0
Beveren 2145.4285714285716
Etterbeek 45.0
Beringen 1118.5714285714284
Dendermonde 795.2857142857142
Vilvoorde 306.8571428571429
Turnhout 800.8571428571429
Deinze* 1820.4285714285716
Dilbeek 588.2857142857143


Nazareth 502.71428571428567
Hemiksem 77.71428571428572
Berlaar 351.0
Alken 402.0
Staden 660.5714285714286
Seneffe 896.7142857142858
Retie 691.2857142857143
Durbuy 2237.2857142857147
Virton 1349.857142857143
Hoeilaart 291.8571428571429
Farciennes 148.42857142857144
Florennes 1907.857142857143
Libramont-Chevigny 2540.857142857143
Moorslede 504.8571428571429
Vosselaar 169.28571428571428
De Panne 341.4285714285714
Ecaussines 496.7142857142858
Grobbendonk 405.1428571428571
Oud-Heverlee 444.85714285714283
Kelmis 258.8571428571429
Ingelmunster 230.85714285714286
Damme 1278.8571428571427
Rebecq 558.2857142857142
Borgloon 730.2857142857142
Ham 466.99999999999994
Meulebeke 419.28571428571433
Borsbeek 56.0
Waasmunster 456.1428571428571
Raeren 1060.142857142857
Tielt-Winge 630.8571428571428
Aiseau-Presles 317.0
Villers-la-Ville 677.8571428571429
De Pinte 256.85714285714283
Kasteelbrakel 324.2857142857143
Court-Saint-Étienne 380.57142857142856
Niel 75.28571428571428
Steenput 453.57142857142856
Blie

,City,Venue,Venue Latitude,Venue Longitude,Category Class,Venue Category
0,Antwerpen,Moochie Frozen Yoghurt,51.220036,4.402850,https://ss3.4sqi.net/img/categories_v2/food/fr...,Frozen Yogurt Shop
1,Antwerpen,Dogma Cocktails,51.221146,4.402854,https://ss3.4sqi.net/img/categories_v2/nightli...,Cocktail Bar
2,Antwerpen,Absinthbar,51.219912,4.400709,https://ss3.4sqi.net/img/categories_v2/nightli...,Cocktail Bar
3,Antwerpen,Pitten en Bonen,51.217657,4.402712,https://ss3.4sqi.net/img/categories_v2/food/ju...,Juice Bar
4,Antwerpen,Kartini Indonesisch Restaurant,51.219270,4.400557,https://ss3.4sqi.net/img/categories_v2/food/in...,Indonesian Restaurant
5,Antwerpen,Hunkemöller,51.218611,4.405531,https://ss3.4sqi.net/img/categories_v2/shops/a...,Lingerie Store
6,Antwerpen,Brasserie Appelmans,51.219879,4.400717,https://ss3.4sqi.net/img/categories_v2/nightli...,Cocktail Bar
7,Antwerpen,Quetzal,51.220625,4.402132,https://ss3.4sqi.net/img/categories_v2/food/co...,Coffee Shop
8,Antwerpen,Maison Tartine,51.221703,4.404996,https://ss3.4sqi.net/img/categories_v2/food/deli_,Sandwich Place
9,Antwerpen,Bia Mara,51.220894,4.400189,https://ss3.4sqi.net/img/categories_v2/food/fi...,Fish & Chips Shop


In [31]:
be_venues.head(10)

,City,Venue,Venue Latitude,Venue Longitude,Category Class,Venue Category
0,Antwerpen,Moochie Frozen Yoghurt,51.220036,4.402850,https://ss3.4sqi.net/img/categories_v2/food/fr...,Frozen Yogurt Shop
1,Antwerpen,Dogma Cocktails,51.221146,4.402854,https://ss3.4sqi.net/img/categories_v2/nightli...,Cocktail Bar
2,Antwerpen,Absinthbar,51.219912,4.400709,https://ss3.4sqi.net/img/categories_v2/nightli...,Cocktail Bar
3,Antwerpen,Pitten en Bonen,51.217657,4.402712,https://ss3.4sqi.net/img/categories_v2/food/ju...,Juice Bar
4,Antwerpen,Kartini Indonesisch Restaurant,51.219270,4.400557,https://ss3.4sqi.net/img/categories_v2/food/in...,Indonesian Restaurant
5,Antwerpen,Hunkemöller,51.218611,4.405531,https://ss3.4sqi.net/img/categories_v2/shops/a...,Lingerie Store
6,Antwerpen,Brasserie Appelmans,51.219879,4.400717,https://ss3.4sqi.net/img/categories_v2/nightli...,Cocktail Bar
7,Antwerpen,Quetzal,51.220625,4.402132,https://ss3.4sqi.net/img/categories_v2/food/co...,Coffee Shop
8,Antwerpen,Maison Tartine,51.221703,4.404996,https://ss3.4sqi.net/img/categories_v2/food/deli_,Sandwich Place
9,Antwerpen,Bia Mara,51.220894,4.400189,https://ss3.4sqi.net/img/categories_v2/food/fi...,Fish & Chips Shop


Write the resulting dataframe to a file

In [22]:
be_venues.to_csv('be_venues.csv',index=False)